In [10]:
#Opomba: Mnozica I predstavlja najvecjo mnozico neodvisnih vozlisc
import time
import random
import csv

# CLP =================================================================================================================================
def clp(graf):
    zac = time.time() #Zacetek merjenja casa

    CLP = MixedIntegerLinearProgram(maximization = True) #CLP, ki ga bomo maximizirali
    vozlisca = CLP.new_variable(binary = True) #Vsakemu vozliscu priredimo vrednost 1 ali 0 (binarne spremenljivke)
    CLP.set_objective(sum([vozlisca[v] for v in graf.vertices()])) #Definiramo kriterijsko funkcijo

    for u,v in graf.edges(labels = False):
        CLP.add_constraint(vozlisca[u] + vozlisca[v] <= 1) #Prvi pogoj CLP

    moc_max = CLP.solve() #resi CLP in nam pove Moc mnozice I
    vozlisca = CLP.get_values(vozlisca) #za vsako vozlisce dobimo njegovo vrednost x_v

    #Seznam vozlisc v mnozici I:
    sez = [k for k, v in vozlisca.items() if v == 1.0]

    #Merjenje casa:
    kon = time.time() #Zaključek merjenja časa
    cas = kon - zac
    return [int(moc_max), sez, cas]


# Relaksacija CLP ==========================================================================================================================
def rclp(graf):
    zac = time.time() #Zacetek merjenja casa

    RCLP = MixedIntegerLinearProgram(maximization = True) #Relak. CLP, ki ga bomo maximizirali
    vozlisca = RCLP.new_variable(real = True) #Vozliščem priredimo vrednosti med 0 in 1
    RCLP.set_max(vozlisca,1)
    RCLP.set_min(vozlisca,0)
    RCLP.set_objective(sum([vozlisca[v] for v in graf.vertices()])) #Kriterijska funkcija

    for u,v in graf.edges(labels = False):
        RCLP.add_constraint(vozlisca[u] + vozlisca[v] <= 1) #Prvi pogoj

    RCLP.solve()
    vozlisca = RCLP.get_values(vozlisca)

    #Seznam vozlisc v mnozici I:
    sez = [k for k, v in vozlisca.items() if v > 0.5] #izberemo tista vozlisca iz slovarja "vozlisca", ki imajo vrednost > 0.5

    #Moc mnozice I:
    moc_max = len(sez)

    #Merjenje casa:
    kon = time.time() #Zaključek merjenja časa
    cas = kon - zac
    return [int(moc_max), sez, cas]


# Lokalno iskanje ==========================================================================================================================
def lokalno_iskanje(graf, st_ponovitev):
    moc_max = 0 #moc največje neodvisne mnozice vozlisc
    max_mnozica_neod = set() #največja mnozica neodvisnih vozlisc
    zac = time.time() #Zacetek merjenja časa

    for i in range(st_ponovitev):
        mnozica_neod = set() #mnozica neodvisnih vozlisc
        mnozica_neod.add(graf.random_vertex()) #dodamo nakljucno vozlisce v mnozica_neod
        for v in random.sample(graf.vertices(), len(graf.vertices())):
            if not any(u in mnozica_neod for u in graf[v]): #Ali je kateri od sosedov vozlišča v množici preverimo
                    mnozica_neod.add(v)
        if len(mnozica_neod) > moc_max: #iscemo najvecjo mnozico neodvisnih vozlisc
            max_mnozica_neod = mnozica_neod
            moc_max = len(mnozica_neod)

    #Merjenje casa:
    kon = time.time() #Zaključek merjenja časa
    cas = kon - zac
    return[moc_max, max_mnozica_neod, cas]

In [11]:
# Funkcija za testiranje vseh treh algoritmov na nakljucnem grafu:
# testiraj(stevilo vozlisc, verjetnost povezave med dvema vozliscema, stevilo ponovitev lokalnega iskanja) =======================================
def testiraj(st_vozlisc, verjetnost, st_ponovitev=10):
    rezultati = []
    graf = graphs.RandomGNP(st_vozlisc, verjetnost)
    rezultati.append(clp(graf))
    rezultati.append(rclp(graf))
    rezultati.append(lokalno_iskanje(graf, st_ponovitev))
    return sum(rezultati, [])#, graf.show(vertex_colors={'red': clp(graf)[1]}) #Seznam seznamov se stakne v en sam seznam

In [12]:
#testiraj(10,0.3,50)

In [13]:
# Funkcija za izpis v CSV v mapo 'Rezultati_testov' (izvede zgornjo funkcijo 'testiraj' na grafih s številom vozlisc od 1 do max_stevilo_vozlisc, pri dolocenih verjetnostih povezave dveh vozlisc in dolocenem stevilu ponovitev lokalnega iskanja):
def izpis_csv(max_stevilo_vozlisc, verjetnost, st_ponovitev):
    #trenutni_cas = time.strftime("%Y-%m-%d_%H-%M-%S")
    with open('Rezultati_testov/test_{0}_{1}_{2}_CLP_cas_3.csv'.format(max_stevilo_vozlisc, verjetnost, st_ponovitev), 'wb') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=';',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        #filewriter.writerow([trenutni_cas])
        filewriter.writerow(['ST_VOZLISC', 'VERJETNOST', 'ST_PONOVITEV_LOKISK',
                             'CLP_MAX_MOC', 'CLP_VOZLISCA_I', 'CLP_CAS', 
                             'RCLP_MAX_MOC', 'RCLP_VOZLISCA_I', 'RCLP_CAS',
                             'LOKISK_MAX_MOC', 'LOKISK_VOZLISCA_I', 'LOKISK_CAS']) #zacena vrstica z imeni stolpcev
        for i in range(1, max_stevilo_vozlisc + 1, 100):
            filewriter.writerow(sum([[i], [verjetnost], [st_ponovitev], testiraj(i, verjetnost, st_ponovitev)], []))

In [14]:
#izpis_csv(2500, 0.3, 50)

In [15]:
#show(graphs.PetersenGraph())
def testirajPetersena(st_ponovitev=10):
    rezultati = []
    graf = graphs.PetersenGraph()
    show(graf)
    rezultati.append(clp(graf))
    rezultati.append(rclp(graf))
    rezultati.append(lokalno_iskanje(graf, st_ponovitev))
    return sum(rezultati, [])#, graf.show(vertex_colors={'red': clp(graf)[1]}) #Seznam seznamov se stakne v en sam seznam

#testirajPetersena(50)

In [16]:
#show(graphs.CubeGraph(4))
def testirajKocko(velikost, st_ponovitev=50):
    rezultati = []
    graf = graphs.CubeGraph(velikost)
    #show(graf)
    rezultati.append(clp(graf))
    rezultati.append(rclp(graf))
    rezultati.append(lokalno_iskanje(graf, st_ponovitev))
    return sum(rezultati, [])#, graf.show(vertex_colors={'red': clp(graf)[1]}) #Seznam seznamov se stakne v en sam seznam

testirajKocko(2, 50)

[0.004635810852050781, 0.0012600421905517578, 0.006648063659667969]

In [1]:
# Funkcija za izpis v CSV v mapo 'Rezultati_testov' (izvede zgornjo funkcijo 'testiraj' na grafih s številom vozlisc od 1 do max_stevilo_vozlisc, pri dolocenih verjetnostih povezave dveh vozlisc in dolocenem stevilu ponovitev lokalnega iskanja):
def izpis_csv_hiperkocka(max_stevilo_vozlisc, st_ponovitev):
    with open('Rezultati_testov/test_hiperkocka_cas.csv', 'wb') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=';',
                                quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for i in range(1, max_stevilo_vozlisc + 1):
            filewriter.writerow(sum([[2^i], testirajKocko(i, st_ponovitev)], []))

izpis_csv_hiperkocka(30, 50)

NameError: global name 'csv' is not defined